# UNet++ 脑肿瘤分割 - 训练Notebook

本notebook用于训练基于UNet++的医学图像分割模型.

## 模型特点:
- UNet++: 改进的UNet架构，具有嵌套的跳跃连接
- 使用segmentation-models-pytorch库
- 支持预训练编码器

## 功能:
1. 从3D体积中提取2D切片
2. 加载和训练UNet++模型
3. 模型评估和可视化
4. 与之前的模型进行对比


## 1. 安装依赖和挂载Google Drive


In [ ]:
# 挂载Google Drive
from google.colab import drive
import time

# 尝试挂载，如果失败则重试
max_retries = 3
retry_count = 0

while retry_count < max_retries:
    try:
        drive.mount('/content/drive', force_remount=False)
        print("Google Drive 挂载成功！")
        break
    except ValueError as e:
        retry_count += 1
        if retry_count < max_retries:
            print(f"挂载失败，{retry_count}/{max_retries} 次重试...")
            print("请确保：")
            print("1. 已点击授权链接并完成授权")
            print("2. 网络连接正常")
            print("3. 等待几秒后重试")
            time.sleep(5)
        else:
            print("挂载失败，请手动运行以下命令：")
            print("from google.colab import drive")
            print("drive.mount('/content/drive')")
            raise


In [ ]:
# 安装必要的包
%pip install segmentation-models-pytorch -q
%pip install nibabel -q
%pip install albumentations -q
%pip install einops -q
%pip install matplotlib -q


## 2. 导入库和配置参数


In [ ]:
# 导入必要的库
import os
import glob
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
from sklearn.model_selection import train_test_split
from collections import defaultdict
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
from datetime import datetime

# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


In [ ]:
# 数据路径配置
DRIVE_DATA_PATH = "/content/drive/MyDrive/data-brain-2024"
MODEL_SAVE_PATH = "/content/drive/MyDrive/brain-tumor-models-unetpp"
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

# 训练参数
IMG_SIZE = 256  # 2D图像尺寸
BATCH_SIZE = 8  # 2D数据可以使用较大的batch size
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50
VAL_INTERVAL = 1
NUM_CLASSES = 4  # 背景 + 3个肿瘤类别

# 预训练编码器选择
PRETRAINED_ENCODER = 'resnet34'  # 可选: resnet34, resnet50, efficientnet-b0等

# 切片提取参数
SLICE_START = 22
NUM_SLICES = 100

print(f"数据路径: {DRIVE_DATA_PATH}")
print(f"模型保存路径: {MODEL_SAVE_PATH}")
print(f"图像尺寸: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"学习率: {LEARNING_RATE}")
print(f"预训练编码器: {PRETRAINED_ENCODER}")


## 3. 数据加载和预处理（提取2D切片）


In [ ]:
def get_patient_groups(data_path):
    """获取所有患者的数据分组"""
    all_files = glob.glob(os.path.join(data_path, "*.nii"))
    patient_groups = defaultdict(lambda: defaultdict(dict))

    for file_path in all_files:
        filename = os.path.basename(file_path)
        match = re.match(r'BraTS-GLI-(\d+)-(\d+)-(t1n|t2f|t2w|t1c|seg)\.nii', filename)
        if match:
            patient_id = match.group(1)
            sequence_id = match.group(2)
            modality = match.group(3)
            patient_groups[patient_id][sequence_id][modality] = file_path

    complete_patients = {}
    for patient_id, sequences in patient_groups.items():
        for seq_id, modalities in sequences.items():
            if 't2f' in modalities and 't1c' in modalities and 'seg' in modalities:
                if patient_id not in complete_patients:
                    complete_patients[patient_id] = {}
                complete_patients[patient_id][seq_id] = modalities

    return complete_patients

def load_nifti_volume(file_path):
    """加载NIfTI文件并返回numpy数组"""
    nii = nib.load(file_path)
    data = nii.get_fdata()
    return data

def extract_slices_from_volume(volume, start_idx=22, num_slices=100):
    """从3D体积中提取2D切片（沿z轴）"""
    depth = volume.shape[2]
    end_idx = min(start_idx + num_slices, depth)
    slices = volume[:, :, start_idx:end_idx]
    return slices

def normalize_slice(slice_data):
    """归一化单个切片"""
    slice_data = slice_data.astype(np.float32)
    max_val = np.max(slice_data)
    if max_val > 0:
        slice_data = slice_data / max_val
    return slice_data

def remap_labels(label_slice):
    """将标签值4映射到3"""
    label_slice = label_slice.astype(np.int64)
    label_slice[label_slice == 4] = 3
    return label_slice

# 获取所有患者数据
all_patient_groups = get_patient_groups(DRIVE_DATA_PATH)
patient_ids = list(all_patient_groups.keys())

print(f"找到 {len(patient_ids)} 个患者")
print(f"前5个患者ID: {patient_ids[:5]}")


In [ ]:
# 准备2D切片数据
def prepare_2d_slice_data(patient_groups, patient_ids):
    """从3D体积中提取2D切片，准备训练数据"""
    slice_data_list = []

    for patient_id in patient_ids:
        if patient_id not in patient_groups:
            continue

        for seq_id, modalities in patient_groups[patient_id].items():
            if 't2f' in modalities and 't1c' in modalities and 'seg' in modalities:
                # 加载3D体积
                t2f_volume = load_nifti_volume(modalities['t2f'])
                t1c_volume = load_nifti_volume(modalities['t1c'])
                seg_volume = load_nifti_volume(modalities['seg'])

                # 提取切片
                t2f_slices = extract_slices_from_volume(t2f_volume, SLICE_START, NUM_SLICES)
                t1c_slices = extract_slices_from_volume(t1c_volume, SLICE_START, NUM_SLICES)
                seg_slices = extract_slices_from_volume(seg_volume, SLICE_START, NUM_SLICES)

                # 为每个切片创建数据项
                num_slices = t2f_slices.shape[2]
                for slice_idx in range(num_slices):
                    slice_data = {
                        't2f_slice': t2f_slices[:, :, slice_idx],
                        't1c_slice': t1c_slices[:, :, slice_idx],
                        'label_slice': seg_slices[:, :, slice_idx],
                        'patient_id': patient_id,
                        'sequence_id': seq_id,
                        'slice_idx': slice_idx
                    }
                    slice_data_list.append(slice_data)

    return slice_data_list

# 准备所有切片数据
all_slice_data = prepare_2d_slice_data(all_patient_groups, patient_ids)
print(f"总共提取了 {len(all_slice_data)} 个2D切片")


In [ ]:
# 按患者ID划分数据（避免数据泄露）
unique_patient_ids = list(set([item['patient_id'] for item in all_slice_data]))
train_patients, temp_patients = train_test_split(
    unique_patient_ids, test_size=0.3, random_state=42
)
val_patients, test_patients = train_test_split(
    temp_patients, test_size=0.5, random_state=42
)

# 根据患者ID划分切片数据
train_slice_data = [item for item in all_slice_data if item['patient_id'] in train_patients]
val_slice_data = [item for item in all_slice_data if item['patient_id'] in val_patients]
test_slice_data = [item for item in all_slice_data if item['patient_id'] in test_patients]

print(f"训练集: {len(train_slice_data)} 个切片 ({len(train_patients)} 个患者)")
print(f"验证集: {len(val_slice_data)} 个切片 ({len(val_patients)} 个患者)")
print(f"测试集: {len(test_slice_data)} 个切片 ({len(test_patients)} 个患者)")
print(f"\n训练患者: {train_patients}")
print(f"验证患者: {val_patients}")
print(f"测试患者: {test_patients}")


## 4. 创建Dataset类（使用Albumentations进行数据增强）


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

class BrainTumor2DDataset(Dataset):
    def __init__(self, slice_data_list, img_size=256, is_train=True):
        self.slice_data_list = slice_data_list
        self.img_size = img_size
        self.is_train = is_train

        # 数据增强（训练时）
        if is_train:
            self.transform = A.Compose([
                A.Resize(img_size, img_size),
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.Rotate(limit=15, p=0.5),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.3),
                A.Normalize(mean=[0.5, 0.5], std=[0.5, 0.5]),
                ToTensorV2()
            ])
            self.label_transform = A.Compose([
                A.Resize(img_size, img_size, interpolation=0),
                ToTensorV2()
            ])
        else:
            # 验证/测试时只做resize和归一化
            self.transform = A.Compose([
                A.Resize(img_size, img_size),
                A.Normalize(mean=[0.5, 0.5], std=[0.5, 0.5]),
                ToTensorV2()
            ])
            self.label_transform = A.Compose([
                A.Resize(img_size, img_size, interpolation=0),
                ToTensorV2()
            ])

    def __len__(self):
        return len(self.slice_data_list)

    def __getitem__(self, idx):
        item = self.slice_data_list[idx]

        # 获取切片
        t2f_slice = normalize_slice(item['t2f_slice'])
        t1c_slice = normalize_slice(item['t1c_slice'])
        label_slice = remap_labels(item['label_slice'])

        # 合并两个模态为2通道图像
        image = np.stack([t2f_slice, t1c_slice], axis=0)  # (2, H, W)
        image = np.transpose(image, (1, 2, 0))  # (H, W, 2)

        # 应用变换
        transformed = self.transform(image=image)
        image = transformed['image']  # (2, H, W)

        # 处理标签
        label_transformed = self.label_transform(image=label_slice)
        label = label_transformed['image'].squeeze(0).long()  # (H, W)

        return {
            'image': image,
            'label': label,
            'patient_id': item['patient_id'],
            'slice_idx': item['slice_idx']
        }

# 创建数据集
train_dataset = BrainTumor2DDataset(train_slice_data, img_size=IMG_SIZE, is_train=True)
val_dataset = BrainTumor2DDataset(val_slice_data, img_size=IMG_SIZE, is_train=False)
test_dataset = BrainTumor2DDataset(test_slice_data, img_size=IMG_SIZE, is_train=False)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print(f"训练集大小: {len(train_dataset)}")
print(f"验证集大小: {len(val_dataset)}")
print(f"测试集大小: {len(test_dataset)}")

# 测试数据加载
sample = train_dataset[0]
print(f"\n样本形状:")
print(f"  图像: {sample['image'].shape}")
print(f"  标签: {sample['label'].shape}")
print(f"  标签值范围: {sample['label'].min().item()} - {sample['label'].max().item()}")


In [ ]:
import segmentation_models_pytorch as smp

# 创建UNet++模型（使用预训练编码器）
model = smp.UnetPlusPlus(
    encoder_name=PRETRAINED_ENCODER,
    encoder_weights='imagenet',  # 使用ImageNet预训练权重
    in_channels=2,  # 2通道：FLAIR + T1CE
    classes=NUM_CLASSES,
    activation=None,  # 使用logits，在loss中处理softmax
)

model = model.to(device)

# 打印模型信息
print(f"模型类型: UNet++")
print(f"编码器: {PRETRAINED_ENCODER}")
print(f"输入通道: 2 (FLAIR + T1CE)")
print(f"输出类别: {NUM_CLASSES}")

# 计算参数量
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n总参数量: {total_params / 1e6:.2f}M")
print(f"可训练参数量: {trainable_params / 1e6:.2f}M")

# 测试前向传播
with torch.no_grad():
    sample_batch = next(iter(train_loader))
    test_input = sample_batch['image'].to(device)
    test_output = model(test_input)
    print(f"\n测试输出形状: {test_output.shape}")
    print(f"期望输出形状: (batch_size, {NUM_CLASSES}, {IMG_SIZE}, {IMG_SIZE})")


## 6. 定义损失函数和优化器


In [ ]:
# 损失函数：Dice Loss + CrossEntropy Loss
class DiceLoss(nn.Module):
    def __init__(self, num_classes=4, smooth=1e-6):
        super().__init__()
        self.num_classes = num_classes
        self.smooth = smooth

    def forward(self, pred, target):
        # pred: (B, C, H, W) - logits
        # target: (B, H, W) - class indices
        pred = torch.softmax(pred, dim=1)

        # 转换为one-hot
        target_one_hot = torch.zeros_like(pred)
        target_one_hot.scatter_(1, target.unsqueeze(1), 1)

        # 计算Dice系数（跳过背景类）
        dice_scores = []
        for c in range(1, self.num_classes):  # 跳过背景
            pred_c = pred[:, c]
            target_c = target_one_hot[:, c]

            intersection = (pred_c * target_c).sum()
            union = pred_c.sum() + target_c.sum()

            dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
            dice_scores.append(dice)

        dice_loss = 1.0 - torch.stack(dice_scores).mean()
        return dice_loss

# 组合损失函数
dice_loss = DiceLoss(num_classes=NUM_CLASSES)
ce_loss = nn.CrossEntropyLoss(ignore_index=-1)

def combined_loss(pred, target):
    dice = dice_loss(pred, target)
    ce = ce_loss(pred, target)
    return 0.5 * dice + 0.5 * ce

# 优化器和学习率调度器
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5
)

print(f"损失函数: Dice Loss + CrossEntropy Loss")
print(f"优化器: AdamW (lr={LEARNING_RATE}, weight_decay=1e-4)")
print(f"学习率调度器: ReduceLROnPlateau")


## 7. 训练和验证函数


In [ ]:
def train_epoch(model, loader, optimizer, loss_function, device):
    model.train()
    epoch_loss = 0

    for batch in tqdm(loader, desc="训练"):
        images = batch['image'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(loader)
    return epoch_loss

def calculate_dice_score(pred, target, num_classes=4, smooth=1e-6):
    """计算Dice系数"""
    pred = torch.softmax(pred, dim=1)
    pred_classes = torch.argmax(pred, dim=1)

    dice_scores = []
    for c in range(1, num_classes):  # 跳过背景
        pred_c = (pred_classes == c).float()
        target_c = (target == c).float()

        intersection = (pred_c * target_c).sum()
        union = pred_c.sum() + target_c.sum()

        if union > 0:
            dice = (2.0 * intersection + smooth) / (union + smooth)
            dice_scores.append(dice.item())

    return np.mean(dice_scores) if dice_scores else 0.0

def val_epoch(model, loader, loss_function, device):
    model.eval()
    val_loss = 0
    all_dice_scores = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="验证"):
            images = batch['image'].to(device)
            labels = batch['label'].to(device)

            outputs = model(images)
            loss = loss_function(outputs, labels)
            val_loss += loss.item()

            # 计算Dice分数
            dice = calculate_dice_score(outputs, labels, NUM_CLASSES)
            all_dice_scores.append(dice)

    val_loss /= len(loader)
    mean_dice = np.mean(all_dice_scores)

    return val_loss, mean_dice

print("训练和验证函数定义完成")


## 8. 开始训练


In [ ]:
# 训练历史
train_losses = []
val_losses = []
val_dice_scores = []

best_val_loss = float('inf')
best_dice_score = 0.0

# 检查是否有检查点
checkpoint_path = os.path.join(MODEL_SAVE_PATH, "checkpoint_latest.pth")
start_epoch = 0

if os.path.exists(checkpoint_path):
    print(f"找到检查点: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    train_losses = checkpoint.get('train_losses', [])
    val_losses = checkpoint.get('val_losses', [])
    val_dice_scores = checkpoint.get('val_dice_scores', [])
    best_val_loss = checkpoint.get('best_val_loss', float('inf'))
    best_dice_score = checkpoint.get('best_dice_score', 0.0)
    print(f"从Epoch {start_epoch}继续训练")
else:
    print("未找到检查点，从头开始训练")

print(f"\n开始训练，共 {NUM_EPOCHS} 个epoch")
print("=" * 60)


In [ ]:
# 训练循环
for epoch in range(start_epoch, NUM_EPOCHS):
    print(f"\nEpoch {epoch + 1}/{NUM_EPOCHS}")
    print("-" * 60)

    # 训练
    train_loss = train_epoch(model, train_loader, optimizer, combined_loss, device)
    train_losses.append(train_loss)

    # 验证
    if (epoch + 1) % VAL_INTERVAL == 0:
        val_loss, mean_dice = val_epoch(model, val_loader, combined_loss, device)
        val_losses.append(val_loss)
        val_dice_scores.append(mean_dice)

        print(f"\n验证结果:")
        print(f"  训练Loss: {train_loss:.4f}")
        print(f"  验证Loss: {val_loss:.4f}")
        print(f"  Dice系数: {mean_dice:.4f}")

        # 更新学习率
        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        print(f"  当前学习率: {current_lr:.6f}")

        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_path = os.path.join(MODEL_SAVE_PATH, "best_model.pth")
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'val_loss': val_loss,
                'dice_score': mean_dice,
            }, best_model_path)
            print(f"  保存最佳模型 (Loss: {val_loss:.4f}, Dice: {mean_dice:.4f})")

        if mean_dice > best_dice_score:
            best_dice_score = mean_dice

    # 保存检查点
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_dice_scores': val_dice_scores,
        'best_val_loss': best_val_loss,
        'best_dice_score': best_dice_score,
    }
    torch.save(checkpoint, checkpoint_path)

print("\n" + "=" * 60)
print("训练完成！")
print(f"最佳验证Loss: {best_val_loss:.4f}")
print(f"最佳Dice系数: {best_dice_score:.4f}")


## 9. 可视化训练历史


In [ ]:
# 绘制训练历史
# 如果变量不存在，从检查点加载
if 'train_losses' not in globals() or 'val_losses' not in globals() or 'val_dice_scores' not in globals():
    print("训练历史变量不存在，尝试从检查点加载...")
    
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)
        train_losses = checkpoint.get('train_losses', [])
        val_losses = checkpoint.get('val_losses', [])
        val_dice_scores = checkpoint.get('val_dice_scores', [])
        print(f"成功从检查点加载训练历史")
    else:
        print("检查点文件不存在")
        train_losses = []
        val_losses = []
        val_dice_scores = []

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(train_losses, label='Training Loss')
axes[0].plot(val_losses, label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True)

if len(val_dice_scores) > 0:
    axes[1].plot(val_dice_scores, label='Validation Dice Score')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Dice Score')
    axes[1].set_title('Validation Dice Score')
    axes[1].legend()
    axes[1].grid(True)
    axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.savefig(os.path.join(MODEL_SAVE_PATH, "training_history.png"), dpi=150, bbox_inches='tight')
plt.show()


## 10. 加载最佳模型并在测试集上评估


In [ ]:
# 加载最佳模型
best_model_path = os.path.join(MODEL_SAVE_PATH, "best_model.pth")
if os.path.exists(best_model_path):
    checkpoint = torch.load(best_model_path, map_location=device, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"加载最佳模型 (Epoch {checkpoint['epoch']}, Dice: {checkpoint['dice_score']:.4f})")
else:
    print("未找到最佳模型，使用当前模型")

# 在测试集上评估
print("\n在测试集上评估...")
test_loss, test_dice = val_epoch(model, test_loader, combined_loss, device)

print(f"\n测试集结果:")
print(f"  Loss: {test_loss:.4f}")
print(f"  平均Dice系数: {test_dice:.4f}")

# 保存测试结果
test_results = {
    'test_loss': test_loss,
    'test_dice_mean': test_dice,
    'model_type': 'UNet++',
    'encoder': PRETRAINED_ENCODER,
    'timestamp': datetime.now().isoformat(),
}

results_path = os.path.join(MODEL_SAVE_PATH, "test_results.json")
with open(results_path, 'w') as f:
    json.dump(test_results, f, indent=2)
print(f"\n测试结果已保存到: {results_path}")


## 11. 训练结果分析


In [ ]:
# 训练结果分析
print("=" * 80)
print("TRAINING RESULTS ANALYSIS REPORT")
print("=" * 80)

print("\n[1. Training Progress]")
print(f"   Completed Epochs: {len(train_losses)}/{NUM_EPOCHS}")
print(f"   Completion: {len(train_losses)/NUM_EPOCHS*100:.1f}%")

print("\n[2. Loss Function Analysis]")
if len(train_losses) > 0:
    print(f"   Training Loss:")
    print(f"      - Initial: {train_losses[0]:.4f}")
    print(f"      - Final: {train_losses[-1]:.4f}")
    improvement = train_losses[0] - train_losses[-1]
    improvement_pct = (improvement / train_losses[0]) * 100
    print(f"      - Improvement: {improvement:.4f} ({improvement_pct:.1f}%)")

if len(val_losses) > 0:
    print(f"\n   Validation Loss:")
    print(f"      - Initial: {val_losses[0]:.4f}")
    print(f"      - Final: {val_losses[-1]:.4f}")
    print(f"      - Best: {best_val_loss:.4f}")

print("\n[3. Dice Coefficient Analysis]")
if len(val_dice_scores) > 0:
    print(f"   Validation Dice Score:")
    print(f"      - Initial: {val_dice_scores[0]:.4f}")
    print(f"      - Final: {val_dice_scores[-1]:.4f}")
    print(f"      - Best: {best_dice_score:.4f}")
    print(f"      - Average: {np.mean(val_dice_scores):.4f}")

print("\n[4. Test Set Performance]")
if 'test_results' in locals():
    print(f"   Test Loss: {test_results['test_loss']:.4f}")
    print(f"   Test Dice Score: {test_results['test_dice_mean']:.4f}")

print("\n[5. Model Configuration]")
print(f"   Model Type: UNet++")
print(f"   Encoder: {PRETRAINED_ENCODER}")
print(f"   Image Size: {IMG_SIZE}x{IMG_SIZE}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Learning Rate: {LEARNING_RATE}")

print("\n" + "=" * 80)
print("Analysis Complete!")
print("=" * 80)
